In [888]:
#encoding=utf-8
import jieba
import numpy as np
import time
from collections import Counter
import sys
import math
doc_count = 0
dict_size = 0
word_count = 0
vocabulary = []
punct_list = []
docs = []
doc_metric_matrix = []
known_words = []
comprehensible_collection = []
frequency_dictionary = {}
comp_max = 0.95
comp_min = 0.90
input_file = "wiki-zh.txt"

#seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
#print("Full Mode: " + "/ ".join(seg_list))  # 全模式


#seg_list = jieba.cut(text, cut_all=False)
#print("Default Mode: " + "/ ".join(seg_list))  # 默认模式

#seg_list = jieba.cut("他来到了网易杭研大厦")
#print(", ".join(seg_list))

#seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
#print(", ".join(seg_list))

In [889]:
def build_chinese_punct_list():
    p_list = []
    with open("input/punct.txt", encoding="utf8") as f:
        for line in f:
            line = line.strip('\n')
            p_list.append(line)
    p_list.append('\n')
    p_list.append('\t')
    return p_list

In [890]:
def strip_chinese_punctuation(str):
    str2 = str.strip('\n')
    str2 = str2.translate({ord(i):' ' for i in punct_list})
    return str2

In [891]:
def build_corpus():
        seg_list = []
        word_list = []
        documents = []
        d_count = 0
        w_count = 0
        file = open(input_file, encoding="utf8")
        N = 1000
        for i in range(N):
            #print(line)
            line = strip_chinese_punctuation(file.readline())
            word_list = []
            seg_list = jieba.cut(line, cut_all=False)
            #print("Segmentation: " + "/ ".join(seg_list))
            for x in seg_list:
                if x != ' ':
                    word_list.append(x)
                    w_count += 1
            documents.append(word_list)
            d_count+=1
        file.close()
        return documents, d_count, w_count

In [892]:
def initialize_doc_metric_matrix():
    doc_count = len(docs)
    #print(doc_count)
    doc_mm = np.zeros((doc_count, 6))
    for i in range(0, doc_count):
        doc_mm[i][0] = i
    return doc_mm
    
#doc_metric_matrix = initialize_doc_metric_matrix()
#print(doc_metric_matrix)

In [893]:
def build_vocabulary():
        v_size = 0
        vocab = list(set(x for l in docs for x in l))
        v_size = len(vocab)
        #print(vocabulary_size)
        return vocab, v_size

In [894]:
def collect_known_words():
    known_list = []
    with open("input/known_vocab.txt", encoding="utf8") as f:
        for line in f:
            line = line.strip('\n')
            known_list.append(line)
    
    #print(known_list)
        #for i in vocabulary:
            #known[col] = sum(1 for k in )
    
    return known_list

In [895]:
def get_doc_word_counts():
    for i in range(0,doc_count):
        doc_metric_matrix[i][1] = len(docs[i])

In [896]:
def build_collection_freq_list():
    freq_dict = {"first":0}
    for i in vocabulary:
        #print(i)
        freq_dict.update({i:0})
        
    for i in range(0,doc_count):
        for j in docs[i]:
            freq_dict[j]+=1
    return freq_dict

In [897]:
def get_doc_known_count():
    known_counter = 0
    known_bool = False
    freq_sum = 0.0
    
    for h in range(0, doc_count):
        doc_word_index = Counter(docs[h])
        #print(doc_word_index[known_words[0]])
        known_counter = 0
        freq_sum = 0.0
        for i in doc_word_index:
            known_bool = False
            for j in known_words:
                #print(i + " " + j + '\n')
                if i == j:
                    known_counter += doc_word_index[i]
                    known_bool = True
                    
            if known_bool == False:
                freq_sum += (-1)/math.log(frequency_dictionary[i]/word_count)
                
        doc_metric_matrix[h][2] = known_counter
        doc_metric_matrix[h][4] = 100*freq_sum/doc_metric_matrix[h][1]
        



In [898]:
def calculate_known_ratio():
    for i in range(0,doc_count):
        doc_metric_matrix[i][3] = doc_metric_matrix[i][2] / doc_metric_matrix[i][1]
    
    #doc_metric_matrix = doc_metric_matrix[np.argsort(cc[:,3])]


In [899]:
def get_comprehensible_collection():
    cc = doc_metric_matrix[((doc_metric_matrix[:,3] > comp_min) & (doc_metric_matrix[:,3] <= comp_max))]
    cc = cc[np.argsort(cc[:,3])]
    return cc

In [900]:
#import csv
#wtr = csv.writer(open ('out.csv', 'w',encoding='utf8'), delimiter=',', lineterminator='\n')
#for x in vocabulary : wtr.writerow ([x])

In [901]:
#def build_term_doc_matrix():
#        row = -1
#        col = -1
#        td_matrix = np.zeros((doc_count, dict_size))
#        print(td_matrix.shape)
#        for i in docs:
#            row += 1
#            for j in vocabulary:
#                col += 1
#                td_matrix[row][col] = sum(1 for k in i if k == j)
#            col = -1
#        return td_matrix"""

In [902]:
punct_list = build_chinese_punct_list()
docs, doc_count, word_count = build_corpus()
vocabulary, dict_size = build_vocabulary()
known_words = collect_known_words()
doc_metric_matrix = initialize_doc_metric_matrix()
get_doc_word_counts()
frequency_dictionary = build_collection_freq_list()
get_doc_known_count()
calculate_known_ratio()
comprehensible_collection = get_comprehensible_collection()


In [903]:
#from wiki_dump_reader import Cleaner, iterate

#cleaner = Cleaner()
#for title, text in iterate(r'C:\Users\Curt\Downloads\zhwiki-20191101-pages-articles-multistream.xml\zhwiki-20191101-pages-articles-multistream.xml'):
#    text = cleaner.clean_text(text)

In [904]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)
comp_min = 0.3
comprehensible_collection = get_comprehensible_collection()
print(comprehensible_collection)

[[  485.     5107.     1533.        0.3       3.452     0.   ]
 [  599.     8538.     2565.        0.3       2.487     0.   ]
 [  914.     2706.      815.        0.301     3.384     0.   ]
 [  404.     1401.      424.        0.303     4.153     0.   ]
 [  545.    12245.     3719.        0.304     2.617     0.   ]
 [  163.      194.       59.        0.304     5.919     0.   ]
 [  192.     7194.     2191.        0.305     3.745     0.   ]
 [  244.      569.      174.        0.306     5.103     0.   ]
 [  538.      251.       77.        0.307     5.859     0.   ]
 [  990.     1005.      309.        0.307     3.86      0.   ]
 [  514.      491.      151.        0.308     5.006     0.   ]
 [  407.      403.      124.        0.308     5.147     0.   ]
 [  418.      469.      145.        0.309     4.35      0.   ]
 [  339.     5494.     1700.        0.309     2.234     0.   ]
 [  377.      470.      146.        0.311     4.432     0.   ]
 [  423.     5278.     1643.        0.311     3.151    

In [905]:
docs[100]

['甘肃省',
 '简称',
 '是',
 '中国',
 '的',
 '一个',
 '省份',
 '位',
 '於',
 '中国',
 '西北地区',
 '黄河',
 '上游',
 '省会',
 '为',
 '兰州市',
 '甘肃',
 '因',
 '甘州',
 '今',
 '张掖',
 '与',
 '肃州',
 '今',
 '酒泉',
 '而',
 '得名',
 '历史',
 '甘肃',
 '为',
 '华夏',
 '文明',
 '的',
 '发祥地',
 '天水',
 '的',
 '伏羲',
 '文化',
 '历史',
 '久远',
 '被',
 '称为',
 '伏羲',
 '故里',
 '在',
 '甘肃',
 '秦安县',
 '发现',
 '的',
 '大地',
 '湾',
 '文化',
 '遗址',
 '被',
 '认为',
 '可能',
 '和',
 '上古时代',
 '的',
 '伏羲',
 '有关',
 '陇东',
 '地区',
 '的',
 '庆阳',
 '一带',
 '是',
 '周人',
 '发祥地',
 '礼县',
 '大',
 '堡子',
 '山有',
 '早期',
 '秦公大墓',
 '秦人',
 '发祥地',
 '西犬丘',
 '就',
 '在',
 '礼县',
 '一带',
 '甘肃',
 '在',
 '春秋时代',
 '东南部',
 '属于',
 '秦国',
 '西部',
 '属于',
 '西戎',
 '公元前',
 '年',
 '在',
 '现在',
 '天水',
 '一带',
 '秦国',
 '设置',
 '了',
 '邽',
 '县',
 '年',
 '设置',
 '陇西',
 '北地',
 '二',
 '郡',
 '汉代',
 '由于',
 '张骞通',
 '西域',
 '河西走廊',
 '成为',
 '丝绸之路',
 '的',
 '必经',
 '要',
 '道',
 '以及',
 '许多',
 '新',
 '的',
 '城镇',
 '兴起',
 '黄河',
 '以东',
 '置',
 '五',
 '郡',
 '统归',
 '凉州',
 '管辖',
 '十六',
 '国',
 '时代',
 '有',
 '个',
 '国家',
 '在',
 '甘肃',
 '境内',
 '建都',
 '南北朝',
 '时代',
 '属',
 '

In [906]:
print(doc_metric_matrix)

[[    0.     2457.      531.        0.216     4.057     0.   ]
 [    1.     4560.     1232.        0.27      3.227     0.   ]
 [    2.     1389.      360.        0.259     4.583     0.   ]
 [    3.     1820.      538.        0.296     4.099     0.   ]
 [    4.     2122.      453.        0.213     3.913     0.   ]
 [    5.     2871.      666.        0.232     3.672     0.   ]
 [    6.     1625.      443.        0.273     4.373     0.   ]
 [    7.     1375.      325.        0.236     4.61      0.   ]
 [    8.     1809.      338.        0.187     4.864     0.   ]
 [    9.     5339.     1210.        0.227     2.939     0.   ]
 [   10.     1394.      322.        0.231     4.579     0.   ]
 [   11.     1996.      375.        0.188     4.19      0.   ]
 [   12.     3891.      849.        0.218     3.952     0.   ]
 [   13.      234.       48.        0.205     5.661     0.   ]
 [   14.     3268.      637.        0.195     3.643     0.   ]
 [   15.     2808.      573.        0.204     3.628    

In [907]:
from operator import itemgetter
sorted(frequency_dictionary.items(), key=itemgetter(1), reverse=True)

[('的', 84537),
 ('在', 23038),
 ('是', 17436),
 ('和', 14562),
 ('年', 13876),
 ('了', 10935),
 ('有', 10211),
 ('為', 7894),
 ('於', 7786),
 ('为', 7383),
 ('也', 7096),
 ('等', 6990),
 ('中', 6295),
 ('被', 5968),
 ('與', 5651),
 ('他', 5552),
 ('後', 5382),
 ('而', 5269),
 ('人', 5025),
 ('上', 4438),
 ('之', 4376),
 ('以', 4334),
 ('与', 4172),
 ('中国', 3953),
 ('由', 3820),
 ('但', 3774),
 ('都', 3420),
 ('不', 3342),
 ('到', 3316),
 ('臺', 3054),
 ('日', 2955),
 ('其', 2756),
 ('后', 2697),
 ('及', 2694),
 ('包括', 2645),
 ('主要', 2557),
 ('于', 2533),
 ('對', 2495),
 ('人口', 2479),
 ('并', 2475),
 ('月', 2451),
 ('其中', 2436),
 ('世界', 2347),
 ('或', 2321),
 ('又', 2281),
 ('中國', 2232),
 ('使用', 2231),
 ('学院', 2222),
 ('日本', 2218),
 ('文化', 2212),
 ('of', 2187),
 ('美国', 2151),
 ('the', 2139),
 ('至', 2107),
 ('就', 2088),
 ('以及', 2076),
 ('可以', 2038),
 ('大', 1909),
 ('其他', 1909),
 ('研究', 1848),
 ('zh', 1846),
 ('对', 1810),
 ('美國', 1784),
 ('政府', 1783),
 ('国家', 1756),
 ('軍', 1751),
 ('來', 1743),
 ('将', 1715),
 ('最', 1694),
 ('个

In [908]:
word_count

2018725

In [909]:
dict_size

234715